# Precompiling Numba modules

One issue with Numba is that it can be hard to install.  With `conda` it's incredibly easy, but not everybody uses `conda` and trying to explain to users/collaborators why they're doing everything wrong is hard.  

Tools like SWIG can compile C/C++ (or other) code at install time and make it available as a Python module if there's some serious numerical heavy-lifting required.  

But if you have ever tried to use SWIG together with NumPy... suffice to say it's a less than ideal arrangement (please don't hurt me, @dabeaz).

## Ahead-of-Time compilation

While Numba's main use is in JIT compiling, they do provide tools for doing AOT compilation.  This pre-compiled module does not rely on Numba, only on NumPy.  (If you are working with collaborators who don't have NumPy installed, I can't help you).

We need to import `numpy`, of course, and also `numba.pycc.CC`

In [1]:
import numpy
from numba.pycc import CC

Name the module `ppe` (I am not creative)

In [2]:
cc = CC('ppe')
cc.verbose = True

In [3]:
@cc.export('pressure_poisson', 
           'f8[:,:](f8[:,:], f8, f8, f8[:,:], f8)')
def pressure_poisson(p, dx, dy, b, l2_target=1e-4):
    pn = p.copy()
    J, I = b.shape
    
    
    iter_diff = l2_target + 1
    
    while iter_diff > l2_target:
        n = 0
        pn = p.copy()
        for j in range(1, J - 1):
            for i in range(1, I - 1):
                p[j, i] = (((pn[j, i + 1] + pn[j, i - 1]) * dy**2 + 
                            (pn[j + 1, i] + pn[j - 1, i]) * dx**2) /
                            (2 * (dx**2 + dy**2)) -
                            dx**2 * dy**2 / (2 * (dx**2 + dy**2)) *
                            b[j, i])

        for j in range(J):
            p[j, 0] = p[j, 1]
            p[j, -1] = p[j, -2]
            
        for i in range(I):
            p[0, i] = p[1, i]
            p[-1, i] = 0
            
        
        if n % 10 == 0:
            iter_diff = (numpy.sum((p - pn)**2)/numpy.sum(pn**2))**.5
        if n == 500:
            break
            
        n += 1
        
    return p


In [4]:
cc.compile()

generating LLVM code for 'ppe' into /tmp/pycc-build-ppe-s9208isc/ppe.cpython-35m-x86_64-linux-gnu.o
C compiler: gcc -pthread -Wsign-compare -Wunreachable-code -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -fPIC

creating /tmp/pycc-build-ppe-s9208isc/home
creating /tmp/pycc-build-ppe-s9208isc/home/gil
creating /tmp/pycc-build-ppe-s9208isc/home/gil/anaconda
creating /tmp/pycc-build-ppe-s9208isc/home/gil/anaconda/lib
creating /tmp/pycc-build-ppe-s9208isc/home/gil/anaconda/lib/python3.5
creating /tmp/pycc-build-ppe-s9208isc/home/gil/anaconda/lib/python3.5/site-packages
creating /tmp/pycc-build-ppe-s9208isc/home/gil/anaconda/lib/python3.5/site-packages/numba
creating /tmp/pycc-build-ppe-s9208isc/home/gil/anaconda/lib/python3.5/site-packages/numba/pycc
creating /tmp/pycc-build-ppe-s9208isc/home/gil/anaconda/lib/python3.5/site-packages/numba/runtime
compile options: '-DPYCC_MODULE_NAME=ppe -DPYCC_USE_NRT=1 -I/home/gil/anaconda/include/python3.5m -I/home/gil/anaconda/lib/python3.5/site-pac

In [5]:
%ls

01.When.where.to.use.Numba.ipynb  06.Compile.Module.ipynb
02.How.Numba.Works.ipynb          0.What.is.Numba.ipynb
04.a.cavity_flow_numpy.ipynb      3.Direct.Summation.ipynb
04.b.cavity_flow_numba_ppe.ipynb  IC.pickle
04.c.cavity_flow_cython.ipynb     numpy_ans.pickle
05.Make.your.own.ufuncs.ipynb     ppe.cpython-35m-x86_64-linux-gnu.so*
06.b.Test.Compiled.Module.ipynb   snippets/
